### 特征统计分析

In [ ]:
%matplotlib inline
import os
import glob
import xgboost
import operator
import numpy as np
from pylab import *
import pandas as pd
import seaborn as sns
import SimpleITK as sitk
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression,Lasso,LassoCV, LassoLarsCV, LassoLarsIC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import cohen_kappa_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectFromModel,RFECV
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, AdaBoostClassifier,VotingClassifier
from sklearn.model_selection import GridSearchCV,KFold, cross_val_predict,StratifiedKFold,train_test_split,cross_val_score,LeaveOneOut
from sklearn.pipeline import make_pipeline
from imblearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score,plot_confusion_matrix,auc,roc_auc_score,roc_curve,plot_roc_curve,ConfusionMatrixDisplay,mean_squared_error
from imblearn.over_sampling import SMOTE,ADASYN
from imblearn.combine import SMOTETomek,SMOTEENN
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler
from sklearn.impute import SimpleImputer
import missingno as msno
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings(action='ignore')

### 导入数据

In [ ]:
df = pd.read_csv('./Selected_features.csv')
scaler = StandardScaler()
#scaler = MinMaxScaler()
df_scaler = scaler.fit_transform(df[df.columns[3:]])
df_scaler=pd.concat([df[df.columns[2:3]], pd.DataFrame(df_scaler,columns=df.columns[3:])], axis=1)#拼接列表
df_scaler

### 绘制箱形图

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='label',y='log-sigma-1-0-mm-3D_firstorder_Kurtosis',hue='stage',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('Nuclear grade', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='stage',y='original_firstorder_Skewness',hue='target',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('original_firstorder_Skewness', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='stage',y='original_glrlm_ShortRunEmphasis',hue='target',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('original_glrlm_ShortRunEmphasis', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='stage',y='original_gldm_DependenceVariance',hue='target',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('original_gldm_DependenceVariance', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='stage',y='original_glszm_GrayLevelNonUniformityNormalized',hue='target',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('original_glszm_GrayLevelNonUniformityNormalized', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
plt.figure(figsize=(6,5))
sns.boxplot(x='stage',y='original_glszm_LargeAreaLowGrayLevelEmphasis',hue='target',data=df_scaler,width=0.5,orient='v')
#plt.ylabel('Nuclear grade', fontsize=12)
plt.xlabel('original_glszm_LargeAreaLowGrayLevelEmphasis', fontsize=12)
plt.xticks(rotation='vertical')
plt.show()

In [ ]:
for i in range(4,104,1):
    plt.figure(figsize=(6,5))
    sns.boxplot(x='stage',y=df.columns[i:i+1][0],hue='target',data=df_scaler,width=0.5,orient='v')
    #plt.ylabel('Nuclear grade', fontsize=12)
    #plt.xlabel('Feature', fontsize=12)
    plt.xticks(rotation='vertical')
    plt.show()

### ap特征分析

In [ ]:
target=df[df.columns[2:3]]
features=df[df.columns[3:]]

In [ ]:
#用selectbest对特征做方差分析F检验
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2,f_classif
model = SelectKBest(f_classif, k=2)
select_feature=model.fit_transform(features, target)
scores=model.pvalues_
indices=np.argsort(scores)[::-1]
new_scores, new_feats = [], []
print('Features ANOVA p_value')
for i in range(len(scores)):
    print("%0.2f %s" % (scores[indices[i]], features.columns[indices[i]]))
    new_scores.append(scores[indices[i]])
    new_feats.append(features.columns[indices[i]])

d = {'features': new_feats, 'p value': new_scores}
p_df = pd.DataFrame(d)
display(p_df)
p_df.to_csv("./P_value.csv", index=False)

In [ ]:
#标准差
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
            % (mean, std * 2, params))

In [ ]:
#计算MCC系数
from sklearn.metrics import matthews_corrcoef
matthews_corrcoef(y_true, y_pred)